# Tema: Simulación de Variables Aleatorias

---

### Ejercicio 1

Dada la siguiente variable aleatoria discreta $X$ con función de probabilidad:

$$
\begin{array}{c|cccc}
x & 1 & 2 & 3 & 4 \\
\hline
P(X = x) & 0.15 & 0.20 & 0.35 & 0.30 \\
\end{array}
$$

Implemente un algoritmo para simular $X$, que minimice el valor esperado del número de iteraciones del algoritmo.  
*(Asuma que no es posible indexar directamente en el array de probabilidades)*

---

### Ejercicio 2

Simule una variable aleatoria con función de densidad:

$$
f(x) = \begin{cases} 
\frac{1}{2}(x - 2), & \text{si } 2 \leq x \leq 3 \\
\frac{1}{2}\left(2 - \frac{x}{3}\right), & \text{si } 3 < x \leq 6 \\
0, & \text{en otro caso.}
\end{cases}
$$

---

### Ejercicio 3

Considere las variables aleatorias $X$, $Y$ y $Z$ independientes y con distribución uniforme, $X \sim U(0,2)$, $Y \sim U(-3,6)$ y $Z$ uniforme sobre el intervalo $(0,1)$.  
Utilizando el método de Monte Carlo estime:

$$
P(Z > XY)
$$

---

### Ejercicio 4

Si se tiene un método para simular variables aleatorias de las distribuciones $F_1$ y $F_2$, proponga un método para simular una variable aleatoria con función de distribución:

$$
F(x) = pF_1(x) + (1 - p)F_2(x), \quad 0 < p < 1.
$$

---

**Nota:** Para los algoritmos de simulación solo tienen permitido usar la variable aleatoria Uniforme $(0,1)$ como conocida (`random.random()` de Python).

In [6]:
import random
import math
random.seed(42)

# Desarrollo

## Índice de ejercicios

- [Ejercicio 1: Simulación de variable discreta](#ejercicio-1-simulación-de-variable-discreta)
- [Ejercicio 2: Simulación de variable continua](#ejercicio-2-simulación-de-variable-continua)
- [Ejercicio 3: Estimación por Monte Carlo](#ejercicio-3-estimación-por-monte-carlo)
- [Ejercicio 4: Mezcla de distribuciones](#ejercicio-4-mezcla-de-distribuciones)

## Ejercicio 3: Estimación por Monte Carlo

### Descripción del problema

El Método de Monte Carlo es una técnica computacional que utiliza la simulación aleatoria para aproximar soluciones a problemas matemáticos o estadísticos que son difíciles o imposibles de resolver analíticamente. En esencia, se basa en la generación de un gran número de muestras aleatorias para modelar un proceso y, a partir de los resultados de esas simulaciones, inferir una propiedad o un valor numérico. La idea central es que al repetir un experimento aleatorio muchas veces, la frecuencia con la que ocurre un evento o el promedio de una cantidad se estabilizará.

La precisión y la validez del Método de Monte Carlo se sustentan directamente en la Ley de los Grandes Números (LGN). Este teorema fundamental de la probabilidad establece que, dado un conjunto de variables aleatorias independientes e idénticamente distribuidas (i.i.d.), la media muestral de estas variables convergerá en probabilidad a la media esperada (o valor esperado) de la distribución subyacente a medida que el número de muestras aumenta.

En el contexto de Monte Carlo, esto significa lo siguiente:

**Estimación de Probabilidades:**  

Si queremos estimar la probabilidad de un evento (por ejemplo, $P(A)$), simulamos el experimento un número $N$ de veces. Contamos cuántas veces el evento $A$ ocurre (digamos $k$ veces). La frecuencia relativa $\frac{k}{N}$ es nuestra estimación de $P(A)$. La LGN garantiza que, a medida que $N \to \infty$,  

$$
\frac{k}{N} \longrightarrow P(A).
$$


**Estimación de Valores Esperados:**  
Si queremos estimar el valor esperado de una función de variables aleatorias, generamos un gran número de realizaciones de esas variables y calculamos el valor de la función para cada realización. El promedio de estos valores de la función es nuestra estimación del valor esperado. La LGN asegura que este promedio muestral convergerá al valor esperado real a medida que el número de simulaciones aumenta.

En resumen, el Método de Monte Carlo traduce un problema determinista o probabilístico complejo en una serie de experimentos aleatorios. La Ley de los Grandes Números proporciona el fundamento teórico que valida este enfoque, asegurando que, con suficientes simulaciones, los resultados obtenidos por medios aleatorios se aproximarán a las soluciones verdaderas del problema.

Aplicaremos el Método de Monte Carlo para estimar la probabilidad $P(Z > X \cdot Y)$ de la siguiente manera:

1. Generaremos un gran número de conjuntos de valores aleatorios $(X_i, Y_i, Z_i)$, donde:  
   - $X_i \sim U(0,2)$  
   - $Y_i \sim U(-3,6)$  
   - $Z_i \sim U(0,1)$  

2. Para cada conjunto, verificaremos si la condición  
   $$Z_i > X_i \cdot Y_i$$  
   se cumple. Cada vez que esto ocurra, contabilizaremos un “éxito”.

3. Estimaremos la probabilidad como  
   $$
   \hat{P} = \frac{\text{número de éxitos}}{N}
   $$
   donde $N$ es el número total de simulaciones.

Gracias a la Ley de los Grandes Números, a medida que aumentemos $N$, $\hat{P}$ convergerá al valor real de $P(Z > X \cdot Y)$.

### Generación de Variables Aleatorias Uniformes a partir de $U(0,1)$

El problema especifica que solo podemos usar la función `random.random()` de Python, que genera números aleatorios de la distribución uniforme estándar $U(0,1)$. Para obtener variables de otras distribuciones uniformes, necesitamos aplicar una **transformación lineal**.

La transformación estándar para generar una variable aleatoria $X \sim U(a,b)$ a partir de una variable $U \sim U(0,1)$ es:

$$
X = a + (b - a) \cdot U
$$

Esta fórmula funciona porque escala el rango de $U$ (que es 1) a la longitud del intervalo deseado $(b-a)$, y luego lo desplaza para que el mínimo sea $a$.

Aplicando esta transformación a nuestras variables:

- Para $X \sim U(0,2)$: aquí $a=0$ y $b=2$.  
  La transformación es:  
  $$
  X = 0 + (2 - 0) \cdot U = 2 \cdot U
  $$
  Por lo tanto, generaremos cada muestra de $X$ como `2 * random.random()`.

- Para $Y \sim U(-3,6)$: aquí $a=-3$ y $b=6$.  
  La transformación es:  
  $$
  Y = -3 + (6 - (-3)) \cdot U = -3 + 9 \cdot U
  $$
  Así, cada muestra de $Y$ se generará como `-3 + 9 * random.random()`.

- Para $Z \sim U(0,1)$: esta variable ya está en el formato deseado, por lo que podemos usarla directamente:  
  $$
  Z = U
  $$
  Es decir, `Z = random.random()`.

### Algoritmo

1. Definir el número de simulaciones $N$ (cuanto mayor, mejor será la estimación).
2. Inicializar un contador `success_count = 0`.
3. Para cada una de las $N$ simulaciones:
    a. Generar $X \sim U(0,2)$ usando `2 * random.random()`.
    b. Generar $Y \sim U(-3,6)$ usando `9 * random.random() - 3`.
    c. Generar $Z \sim U(0,1)$ usando `random.random()`.
    d. Evaluar la condición $Z > X \cdot Y$. Si es verdadera, incrementar `success_count`.
4. La probabilidad estimada es $\displaystyle \frac{\text{success\_count}}{N}$.

### Código

In [7]:
# Definir el número de simulaciones N
num_simulaciones_N = 1000000 # Un millón de simulaciones para una buena estimación

# Inicializar un contador para los éxitos
success_count = 0

# Para cada una de las N simulaciones:
for _ in range(num_simulaciones_N):
    # a. Generar X ~ U(0,2)
    X = 2 * random.random()
    
    # b. Generar Y ~ U(-3,6)
    Y = 9 * random.random() - 3
    
    # c. Generar Z ~ U(0,1)
    Z = random.random()
    
    # d. Evaluar la condición Z > X * Y. Si es verdadera, incrementar success_count.
    if Z > X * Y:
        success_count += 1

# La probabilidad estimada es success_count / N
probabilidad_estimada = success_count / num_simulaciones_N

print("\n--- Estimación de P(Z > XY) por Monte Carlo ---")
print(f"Número de simulaciones (N): {num_simulaciones_N}")
print(f"Estimación de P(Z > XY): {probabilidad_estimada:.6f}")


--- Estimación de P(Z > XY) por Monte Carlo ---
Número de simulaciones (N): 1000000
Estimación de P(Z > XY): 0.444010


## Ejercicio 4: Mezcla de distribuciones

### Descripción del problema
El **método de composición** es una técnica fundamental en la simulación de variables aleatorias, especialmente útil cuando la función de distribución acumulada (CDF) objetivo, $F(x)$, puede expresarse como una combinación convexa (o mezcla ponderada) de otras distribuciones más sencillas, $F_i(x)$. 

En este ejercicio, la función de distribución dada es:

$$
F(x) = p F_1(x) + (1 - p) F_2(x)
$$

donde $F_1(x)$ y $F_2(x)$ son las CDFs de dos distribuciones y $p$ es el peso asignado a $F_1(x)$, con $0 < p < 1$.

La esencia de este método radica en simular la variable aleatoria en dos pasos:

1. Se toma una decisión probabilística:  
   - Con probabilidad $p$, elegimos simular la distribución $F_1$.
   - Con probabilidad $1-p$, elegimos simular la distribución $F_2$.
2. Una vez seleccionada una de las distribuciones componentes, se genera una muestra a partir de esa distribución específica utilizando los métodos de simulación disponibles para $F_1$ y $F_2$.

Este enfoque se basa directamente en la definición de una **distribución de mezcla**. La probabilidad de que una muestra generada por este método sea menor o igual a $x$ es precisamente la suma ponderada de las probabilidades de cada componente:

$$
P(X \leq x) = P(X \leq x \mid \text{se eligió } F_1) \cdot P(\text{se eligió } F_1) + P(X \leq x \mid \text{se eligió } F_2) \cdot P(\text{se eligió } F_2)
$$

$$
P(X \leq x) = F_1(x) \cdot p + F_2(x) \cdot (1 - p)
$$

Lo cual es exactamente la CDF $F(x)$ deseada. Esto demuestra que el método de composición es matemáticamente sólido y nos permite simular a partir de una distribución compleja a través de la simulación de sus componentes más simples. Además, evita la necesidad de invertir analítica o numéricamente la CDF de la mezcla, lo cual suele ser un proceso mucho más desafiante.

### Algoritmo

Para simular una variable aleatoria $X$ con función de distribución $F(x) = p F_1(x) + (1 - p) F_2(x)$:

1. **Generar un número aleatorio uniforme:**
    - Obtener un valor $U \sim U(0,1)$ usando `random.random()`.

2. **Decidir de qué componente simular:**
    - Si $U < p$:
        - Simular una variable aleatoria de la distribución $F_1$.
    - Si $U \geq p$:
        - Simular una variable aleatoria de la distribución $F_2$.

3. El valor resultante de la simulación en el paso 2 será la muestra de la variable aleatoria con función de distribución $F(x)$.



### Código

In [8]:
def simulate_exp(lambda_rate):
    """
    Simula una variable aleatoria Exponencial(lambda) usando el método de la transformada inversa.
    """
    u = random.random()
    return -1 / lambda_rate * math.log(u)

def simulate_mixture_F(p, lambda1, lambda2):
    """
    Simula una variable aleatoria de una distribución de mezcla F(x) = pF1(x) + (1-p)F2(x).
    Aquí, F1 y F2 son distribuciones Exponenciales.
    """
    u_choice = random.random()
    if u_choice < p:
        # Simular de F1 (Exponencial con lambda1)
        return simulate_exp(lambda1)
    else:
        # Simular de F2 (Exponencial con lambda2)
        return simulate_exp(lambda2)

# Ejemplo de simulación
print("\n--- Simulación de Variable Aleatoria de Distribución de Mezcla ---")
p_val = 0.4      # Probabilidad de elegir F1
lambda_1 = 0.5   # Tasa para F1 (Exp(0.5))
lambda_2 = 2.0   # Tasa para F2 (Exp(2.0))

print(f"Parámetros: p={p_val}, lambda1={lambda_1}, lambda2={lambda_2}")
print("Ejemplo de 10 simulaciones de la variable de mezcla:")
simulated_data_mixture = [simulate_mixture_F(p_val, lambda_1, lambda_2) for _ in range(10)]
print(simulated_data_mixture)


--- Simulación de Variable Aleatoria de Distribución de Mezcla ---
Parámetros: p=0.4, lambda1=0.5, lambda2=2.0
Ejemplo de 10 simulaciones de la variable de mezcla:
[0.5957776315931858, 0.21402775041186964, 0.1287652780963939, 2.6636013429081333, 0.35712671083669995, 0.4275816546090624, 1.9604850528847382, 2.2510418976254525, 0.4623230576385686, 1.172476660652432]
